# 05 – Image-based Weather Severity Classifier

## Objectives

- Build a **lightweight image-based weather severity classifier** to support the
  winter tour demand & cancellation project.
- Use **hand-crafted features** extracted from mountain weather images (brightness,
  colourfulness, proportion of bright pixels, etc.) rather than heavy CNN frameworks.
- Train a small scikit-learn model to classify images into **mild / moderate / severe**
  weather categories.

## Inputs

- Labelled images stored under:
  - `data/raw/weather_images/mild/`
  - `data/raw/weather_images/moderate/`
  - `data/raw/weather_images/severe/`
- Feature extraction logic from `src/image_features.py`.

## Outputs

- A trained scikit-learn classifier predicting `weather_severity` from image features.
- Saved model file: `models/weather_severity_model.pkl`.
- Basic evaluation metrics (accuracy, confusion matrix) to validate that the model
  is good enough as a **prototype** to support weather-based decision-making in the app.


In [ ]:
import os
from pathlib import Path
from typing import List, Dict

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import joblib

from src.image_features import extract_weather_features_from_bytes


In [ ]:
# Base project directory (adjust if needed)
BASE_DIR = Path(__file__).resolve().parents[1]

DATA_RAW = BASE_DIR / "data" / "raw" / "weather_images"
MODEL_PATH = BASE_DIR / "models" / "weather_severity_model.pkl"

# Simple label mapping: folder name -> class label
LABELS = ["mild", "moderate", "severe"]

DATA_RAW, MODEL_PATH, LABELS
